In [ ]:
import timm 
from fastai.vision.all import * 
from pathlib import Path

: 

In [ ]:
path = Path('fluffy or not')
dls = DataBlock(
    blocks=(ImageBlock, CategoryBlock),
    get_items=get_image_files,
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=[Resize(192, method='squish')]
).dataloaders(path, bs=60)
dls.show_batch(max_n=6)

In [ ]:
control_learn = vision_learner(dls, '', metrics=error_rate)
control_learn.fine_tune(3)

In [ ]:
accuracy_learn = vision_learner(dls, '', metrics=error_rate)
accuracy_learn.fine_tune(3)

In [ ]:
speed_learn = vision_learner(dls, '', metrics=error_rate)
speed_learn.fine_tune(3)